- synchronize the dimention of the nodes embeddings (dim =4, each) with the input of the server model.
- handle the final round to save automaticlly the encoders/weights.
- handle the transfert of encoders/weights and preprocessors automaticlly to the server result folder.
- match model architecture between trainning and load for prediction.

we need to get the entire dataset, and extract the relative vital_signs and lab_results,then align them.(all need to be aligned due to VFL logic) And distibute them across nodes.

In [ ]:
import functions

In [ ]:
csv_paths = ['../data/node3_chartevents/data.csv','../data/node1_patients/data.csv']
original_dataframe = functions.merge_csvs_on_feature(csv_paths=csv_paths,merge_on='subject_id')

In [ ]:
_ = functions.analyze_dataframe(original_dataframe)

- After checking and analysing the features within the dataframe, we choose the feature with less missingness to apply the regression on it, and choose also within the next steps the most related features that can explain it. 

In [ ]:
selected_feature = 'respiratory_rate'
functions.calculate_mutual_information(original_dataframe,selected_feature)

In [ ]:
functions.calculate_spearman_correlation(original_dataframe,selected_feature)

The next step is to test correlation (linear and !linear) between features to apply the specific algorithm to the specific features, but the problem is that features are distributed across nodes within the VFL environment, so to test correlation we need to creat an other system that can handle this step with privacy preserving.(tip used : we gonna based on the HFL correlations tests.)

In [ ]:
target_column = 'respiratory_rate'
features = ['respiratory_rate','anchor_year','heart_rate']
cleaned_dataframe = functions.prepare_clean_dataset(original_dataframe,features=features)

In [ ]:
_= functions.analyze_dataframe(cleaned_dataframe)

In [ ]:
missing_rate = 0.2
pattern = 'MAR'

data_with_missingness , original_values = functions.introduce_missingness(df=cleaned_dataframe,feature1=features[0],feature2=features[1],missing_rate=missing_rate,task="regression",pattern=pattern)
original_values

In [ ]:
_= functions.analyze_dataframe(data_with_missingness)

In [ ]:
x= functions.load_ml_model('../server/ml_regression')

In [ ]:
df = functions.predict_with_ml_model(data_with_missingness,'ml_r')

In [ ]:
df